In [ ]:
using Pkg
Pkg.activate("CT")
using CT
# benchmark adder_MPO
using ITensors
# initialize random state
using .CT: CT_MPS
using Random
using ProgressMeter

L = 20
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
_maxdim = 2^(div(L,2))
_maxdim0 = 50
_eps = 0.0
seed = 123457
x0 = nothing
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
ct_h = CT_MPS(L=L,xj=xj,folded=folded,_maxdim=_maxdim,ancilla=ancilla,seed_vec=seed_vec,seed=seed,x0=x0,debug=false);
ct_t = CT_MPS(L=L,xj=xj,folded=folded,_maxdim=_maxdim,ancilla=ancilla,seed_vec=seed_vec,seed=seed,x0=x0,debug=false,passthrough=true);

i = 1
j = 1
p_ctrl = 0.4
p_proj = 0.7
ancilla = 0
T_max = ancilla ==0 ? 2*(ct_h.L^2) : div(ct_h.L^2,2)
# Write a single string
for idx in 1:T_max
    @time i = CT.random_control!(ct_h,i,p_ctrl,p_proj)
    @time j = CT.random_control!(ct_t,j,p_ctrl,p_proj)
end

  Activating project at `~/CT_MPS_mini/CT`


MersenneTwister(123457)

In [6]:
include("global_adder_passthrough.jl")
adder_mpo_passthrough_vec = adder_mpo_vec(L, xj, qubit_site, ancilla, folded, phy_list, phy_ram, _maxdim);

bond dims before fix spurs: Any[2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3] Any[2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3]
bond dims after fix spurs: Any[4, 8, 16, 16, 16, 8, 8, 8, 8, 8, 8, 8, 8, 6, 3]
bond dims before fix spurs: Any[2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3] Any[2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3]
bond dims after fix spurs: Any[2, 7, 8, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8, 6, 3]
bond dims before fix spurs: Any[3, 4, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3] Any[3, 4, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3]
bond dims after fix spurs: Any[4, 14, 8, 8, 4, 4, 8, 8, 8, 8, 8, 8, 8, 6, 3]
bond dims before fix spurs: Any[3, 4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 3] Any[3, 4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 3]
bond dims after fix spurs: Any[4, 10, 22, 24, 8, 8, 4, 4, 8, 8, 8, 8, 8, 6, 3]
bond dims before fix spurs: Any[3, 4, 4, 4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 3] Any[3, 4, 4, 4, 4, 4, 2, 2, 4, 4, 4, 4, 4, 4, 3]
bond dims after fix spurs: Any[3, 6, 16, 16, 24, 24, 8, 8, 

In [7]:
typeof(adder_mpo_passthrough_vec)

Vector{MPO} (alias for Array{MPO, 1})

In [8]:
adder_mpo_passthrough_vec[1]

MPO
[1] ((dim=2|id=86|"Qubit,Site,n=1")', (dim=2|id=86|"Qubit,Site,n=1"), (dim=4|id=485|"Link,l=1"))
[2] ((dim=2|id=975|"Qubit,Site,n=2")', (dim=2|id=975|"Qubit,Site,n=2"), (dim=8|id=953|"Link,l=2"), (dim=4|id=485|"Link,l=1"))
[3] ((dim=2|id=676|"Qubit,Site,n=3")', (dim=2|id=676|"Qubit,Site,n=3"), (dim=16|id=244|"Link,l=3"), (dim=8|id=953|"Link,l=2"))
[4] ((dim=2|id=703|"Qubit,Site,n=4")', (dim=2|id=703|"Qubit,Site,n=4"), (dim=16|id=661|"Link,l=4"), (dim=16|id=244|"Link,l=3"))
[5] ((dim=2|id=231|"Qubit,Site,n=5")', (dim=2|id=231|"Qubit,Site,n=5"), (dim=16|id=886|"Link,l=5"), (dim=16|id=661|"Link,l=4"))
[6] ((dim=2|id=26|"Qubit,Site,n=6")', (dim=2|id=26|"Qubit,Site,n=6"), (dim=8|id=900|"Link,l=6"), (dim=16|id=886|"Link,l=5"))
[7] ((dim=2|id=947|"Qubit,Site,n=7")', (dim=2|id=947|"Qubit,Site,n=7"), (dim=8|id=203|"Link,l=7"), (dim=8|id=900|"Link,l=6"))
[8] ((dim=2|id=773|"Qubit,Site,n=8")', (dim=2|id=773|"Qubit,Site,n=8"), (dim=8|id=954|"Link,l=8"), (dim=8|id=203|"Link,l=7"))
[9] ((dim=2|i

In [9]:
for i1 in 1:L
    initial_state = randomMPS(MersenneTwister(nothing), qubit_site, linkdims=_maxdim0)
    final_1 = apply(adder_mpo_passthrough_vec[i1], initial_state; cutoff=_eps, maxdim=_maxdim)
    final_2 = apply(adder[i1],initial_state; cutoff=_eps, maxdim=_maxdim)
    println(norm(final_1-final_2))
end


1.275866029520122e-11
1.5274538231053338e-11
1.2209194061411162e-11
1.278870439758224e-11
3.331722953807532e-12
2.8452452908781712e-12
1.0025931642637128e-11
3.4740354883356695e-12
4.30855677833388e-11
2.9793979829263115e-11
1.45874420960319e-11
1.5293102598905416e-13
2.515774479904966e-11
4.637908589181953e-12
5.8092688775815266e-12
6.798606910786922e-12


In [29]:
# # conn_pairs_dict[i1_test], folded_ram_phy_dict[i1_test]
# i1 = 5
# dict_correct = Dict{String,String}()
# for state_index in 1:2^L
#     string_vec = lpad(string(state_index-1,base=2),L,"0")
#     correct_unfolded = lpad(string((state_index - 1 + shift_amount) % 2^L,base=2),L,"0")
#     ram_phy = folded_ram_phy_dict[i1]
#     string_vec_folded = join([string_vec[ram_phy[i]] for i in 1:L])
#     correct_folded = join([correct_unfolded[ram_phy[i]] for i in 1:L])
#     dict_correct[join(string_vec)] = join(correct_folded)
# end


# @showprogress for state_index in 1:2^L
#     string_vec = lpad(string(state_index-1,base=2),L,"0")
#     string_vec_folded = join([string_vec[ram_phy[i]] for i in 1:L])
#     # Convention: RAM position i stores physical bit ram_phy[i]
#     # productMPS assigns to RAM positions, so use folded representation
#     vec = 1 .+ parse.(Int, [string(string_vec_folded[i]) for i in 1:L])
#     initial_state = productMPS(qubit_site, vec)

#     # @show CT.mps_element(initial_state, "000000001000"[ram_phy])
#     final_state_1 = copy(initial_state)
#     final_state_2 = copy(initial_state)
#     # passthrough_state = copy(initial_state)
#     final_state_1 = apply(adder_mpo_passthrough_vec[i1], final_state_1; cutoff=_eps, maxdim=_maxdim);
#     # passthrough_state = global_adder(passthrough_state, carry_links, T_vec, gate_vec, qubit_site; cutoff=_eps, maxdim=_maxdim);
#     final_state_2 = apply(adder_mpo_vec_haining[i1],final_state_2; cutoff=_eps, maxdim=_maxdim);
#     final_state_1 = display_state(final_state_1)
#     final_state_2 = display_state(final_state_2)
#     if final_state_1 != final_state_2
#         println(string_vec, "=>", final_state_1, "<=>", final_state_2)
#     end
#     # println(string_vec, "=>", display_state(final_state_1), "<=>", display_state(final_state_2))
#     # println(display_state(final_state_1) == display_state(final_state_2))
# end